<a href="https://colab.research.google.com/github/illhyhl1111/AI_Expert_2021/blob/main/Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

import pdb
from torch.utils.tensorboard import SummaryWriter
import os
import datetime
%tensorflow_version 1.x

#### Connect to local google drive & settings for export/import models

In [ ]:
drive.mount('/content/gdrive')
!mkdir ./gdrive/'My Drive'/MNIST_models/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
mkdir: cannot create directory ‘./gdrive/My Drive/MNIST_models/’: File exists


#### hyperparameters & datasets

In [ ]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30
IS_DNN = False
DEBUG = False
USE_TENSORBOARD = False

In [ ]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

#### CNN model

In [ ]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.out_size = 8*2*2
        self.fc = nn.Linear(self.out_size,10)
    
    def forward(self,x):
        if DEBUG:
            pdb.set_trace()
        x = self.BN1(F.relu(self.conv1(x)))
        x = self.BN2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1,self.out_size)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [ ]:
def compute_acc(y_,y):
    # pdb.set_trace()
    _, argmax = torch.max(y_,dim=1)
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

## pdb commands
* `n(ext)`: 이번 line 실행(함수 내부로 들어가지 않음)
* `s(tep)`: 이번 line 실행(함수 내부로 들어감)
* `r(eturn)`: 현재 함수의 return이 실행될때까지 계속 실행
* `l(ist)`: 현재 line과 위아래 5줄 출력
* `c(ontinue)`: 다음 breakpoing(set_trace)가 실행될때까지 계속 실행
* `p <variable>`: 변수의 값 출력
* 그 외 python interpreter를 사용하는 것 처럼 아무런 명령어 실행 가능
  * ex) `x += 3` -> 변수 x의 현재 값에 3이 더해짐
  * `a += 1` 과 같이 변수가 pdb command와 이름이 겹칠 경우, `!`를 넣고 실행
    * ex) `!a += 1`

#### Training part(CNN)

In [ ]:
DEBUG = False

In [ ]:
if not IS_DNN:
    model = MNISTCNN(IMG_SIZE).cuda()

    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    num_params = sum([np.prod(p.size()) for p in model_parameters])
    print("number of parameters : {}".format(num_params))
    
    optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss()

    i_step = 0
    for epoch in range(NUM_EPOCHES):
        tot_loss = 0.0
        
        for idx, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            x = x.cuda()
            y_ = model(x)
            loss = loss_fn(y_, y.cuda())
            loss.backward()
            tot_loss+=loss.item()
            optimizer.step()

            if USE_TENSORBOARD and idx % 10 == 0:
                writer.add_scalar('loss', loss.item(), i_step)
                i_step += 1
            
        print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))

        model.eval()
        
        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        test_acc = compute_acc(y_,y.numpy())
        
        print("Acc(test) : {}".format(test_acc))
        if USE_TENSORBOARD:
            writer.add_scalar('accuracy', test_acc, epoch)
        
        model.train()

    torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 430.27998661994934
Acc(test) : 0.97265625
Epoch 2, Loss(train) : 362.74468219280243
Acc(test) : 0.953125
Epoch 3, Loss(train) : 356.19120597839355
Acc(test) : 0.97265625
Epoch 4, Loss(train) : 353.686425447464
Acc(test) : 0.984375
Epoch 5, Loss(train) : 352.28338301181793
Acc(test) : 0.96875
Epoch 6, Loss(train) : 351.37072145938873
Acc(test) : 0.9765625
Epoch 7, Loss(train) : 350.75315952301025
Acc(test) : 0.9765625
Epoch 8, Loss(train) : 350.00396275520325
Acc(test) : 0.9921875
Epoch 9, Loss(train) : 349.6397304534912
Acc(test) : 0.984375
Epoch 10, Loss(train) : 349.20270466804504
Acc(test) : 0.98046875
Epoch 11, Loss(train) : 348.8227311372757
Acc(test) : 0.98046875
Epoch 12, Loss(train) : 348.6930192708969
Acc(test) : 0.97265625
Epoch 13, Loss(train) : 348.4191929101944
Acc(test) : 0.98046875
Epoch 14, Loss(train) : 348.15966069698334
Acc(test) : 0.97265625
Epoch 15, Loss(train) : 348.04023110866547
Acc(test) : 0.96875
Epoch 16, Lo

In [ ]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)

test_acc = compute_acc(y_,y.numpy())

print("Acc(test) : {}".format(test_acc))

#### initialize SummaryWriter
* tensorboard 사용을 위해서 커맨드로 tensorboard를 미리 실행해야 함
  * `tensorboard --logdir=path/to/log-directory`
  
* colab에서 tensorboard 실행  
  * `%load_ext tensorboard`
  * `%tensorboard --logdir path/to/log-directory`

In [ ]:
now = datetime.datetime.now()
now_str = now.strftime('%m-%d-%H-%M-%S')
rundir = '/content/gdrive/My Drive/MNIST_models/runs'
logdir = os.path.join(rundir, now_str)
os.makedirs(rundir, exist_ok=True)
writer = SummaryWriter(logdir)

In [ ]:
USE_TENSORBOARD = True

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/gdrive/My Drive/MNIST_models/runs'